In [1]:
import sys, os, re, time
import numpy as np
import pandas as pd
import defusedxml.ElementTree as ET
from collections import defaultdict
# import xml.etree.ElementTree as ET
from loadconfig import CONFIG

if '../cpp/fit-build' not in sys.path: 
    sys.path.append('../cpp/fit-build')
    
from pyfitparquet_so import FPTransformer

In [ ]:
timestamps = [pd.to_datetime(t).tz_localize(None) for t in 
              ['2021-01-13T20:05:05Z', '2021-01-13T20:05:06Z', 
               '2021-01-13T20:05:09Z']]

df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', None],
                   'three': [True, False, True],
                   'four': timestamps},
                   index=list('abc'))

In [ ]:
display(df)
# df.to_parquet(path='example.parquet', engine='pyarrow')
print(df.dtypes)


In [ ]:
def recurse_tree(xiter,  pad=''):
#{
    xnode = next(xiter)
    tagname = xnode.tag
    print(pad + tagname + " " + str(xnode.attrib))

    for _ in range(len(xnode)):
        recurse_tree(xiter, f'  {pad}')
#}

# Deal w/Clark notation
def striptag(clarkname):
    # Tcx xnode.tag generally includes xmlns, thus appearing as: '{xmlns}tagname'
    matchobj = re.match(r'^(\{(?P<xmlns>\w+:.+)\})?(?P<tagname>\w+)', clarkname)
    assert matchobj, f"Unable to appropriately strip element tag: '{clarkname}'"
    return matchobj.group('xmlns'), matchobj.group('tagname')

def analyze_tree(xiter, compoundtags=None, tagstack=None):
#{
    #if tagmap is None: tagmap = set()
    #if attribmap is None: attribmap = defaultdict(set)
    
    if tagstack is None: tagstack = []
    if compoundtags is None: compoundtags = set()

    xnode = next(xiter)
    xmlns, tagname = striptag(xnode.tag)
    
    tagstack.append(tagname)
    comptag = '-'.join(tagstack)
    for k in xnode.attrib.keys():
        xmlns, kname = striptag(k)
        compoundtags.add(f'{comptag}-{kname}')
        
    if xnode.text and xnode.text.rstrip(): compoundtags.add(comptag)
        
    for _ in range(len(xnode)):
        analyze_tree(xiter, compoundtags, tagstack)
    
    tagstack.pop()
#}

In [ ]:
# xiter = ET.parse('example.xml').iter()
xiter = ET.parse('../../data/TeamScream.tcx').iter()

compoundtags = set()
analyze_tree(xiter, compoundtags)
compoundtags

In [ ]:
CONFIG

In [4]:
parquet_maker = PyFitParquet()
#parquet_maker.data_to_parquet('../../data/')
df = parquet_maker.tcx_to_parquet('../../data/TeamScream.tcx', './TeamScream.parquet')
display(df)
display(df[:60])

,source_filename,manufacturer_name,product_index,timestamp,mesg_name,field_index,field_name,field_type,value_string,value_float,units
0,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:10,Activity,None,sport,string,Biking,NaN,None
1,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:10,Lap,None,average_heart_rate,integer,178,178.00,bpm
2,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:10,Lap,None,calories,integer,913,913.00,kcal
3,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:11,Lap,None,distance,float,38175.74,38175.74,m
4,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:11,Lap,None,intensity,string,Active,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
20760,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 17:49:39,Creator,None,unit_id,integer,1,1.00,None
20761,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 17:49:39,Creator,None,build_major,integer,0,0.00,None
20762,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 17:49:39,Creator,None,build_minor,integer,0,0.00,None
20763,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 17:49:39,Creator,None,version_major,integer,1,1.00,None


,source_filename,manufacturer_name,product_index,timestamp,mesg_name,field_index,field_name,field_type,value_string,value_float,units
0,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:10,Activity,None,sport,string,Biking,NaN,None
1,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:10,Lap,None,average_heart_rate,integer,178,178.000000,bpm
2,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:10,Lap,None,calories,integer,913,913.000000,kcal
3,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:11,Lap,None,distance,float,38175.74,38175.740000,m
4,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:11,Lap,None,intensity,string,Active,NaN,None
5,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:11,Lap,None,maximum_heart_rate,integer,199,199.000000,bpm
6,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:12,Lap,None,maximum_speed,float,10.639552,10.639552,m/s
7,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:12,Lap,None,total_time,integer,4169,4169.000000,s
8,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:12,Lap,None,trigger_method,string,Manual,NaN,None
9,TeamScream.tcx,The Sufferfest Training System,1,2021-01-09 16:40:13,Trackpoint,None,distance,float,1.88,1.880000,m


In [ ]:
pfile = './TeamScream.parquet'

pdframe = pd.read_parquet(pfile, engine='pyarrow')
print(pfile + ":")
display(pdframe)

In [ ]:
#     def append_position(self, xiter, nlen):
#         for _ in range(nlen):
#             fnode = next(xiter)
#             schema, tagname = self.striptag(fnode)
#             if tagname == "LatitudeDegrees": 
#             elif tagname == "LongitudeDegrees": 

#         <xsd:element name="LatitudeDegrees" type="DegreesLatitude_t"/>
#         <xsd:element name="LongitudeDegrees" type="DegreesLongitude_t"/>
        
#     def insert_altitude
#     def insert_distance
#     def insert_heart_rate
#     def insert_cadence
#     def insert_sensor_state
#     def insert_extensions

def append_trackpoint(self, xiter, nlen):
    #{
        nfields = 0
        for ti in range(nlen):
            fnode = next(xiter)
            xmlns, tagname = striptag(fnode)
            if tagname == "Time":
                self.timestamp = pd.to_datetime(fnode.text).tz_localize(None)
                if self.excludeflags["exclude_timestamp_values"]: continue
                append_mesg_fields('Trackpoint'):
                append_field_fields()
                nfields += 1
                
                
#             elif tagname == "Position":
#                 for pi in range(len(fnode)):
#                     fnode = next(xiter)
#                     xmlns, tagname = self.striptag(fnode)
#                     if tagname == "LatitudeDegrees": 
#                     elif tagname == "LongitudeDegrees": 
                
#             elif tagname == "AltitudeMeters": 
#             elif tagname == "DistanceMeters":
#             elif tagname == "HeartRateBpm":
#             elif tagname == "Cadence":
#             elif tagname == "SensorState":
#             elif tagname == "Extensions":
                
            print("  " + tagname + " : " + fnode.text.rstrip())
    #}
    


# try: 
# #{
#     xiter = ET.parse(tcx_fname).iter()

#     # TODO: find/get Creator tag first to init "file_id" fields

#     debugcounter = 0
#     while True:
#         xnode = next(xiter)
#         schema, tagname = striptag(xnode)

#         print(tagname)
#         if tagname == 'Trackpoint':
#             append_trackpoint(xiter, len(xnode))
#         elif tagname == 'Lap':
#             append_lap(xiter, len(xnode))

#         # Tmp loop breaker
#         if debugcounter > 200: break
#         debugcounter += 1
# #}
# except StopIteration: pass

In [ ]:
tree = ET.parse('example.xml')
for elem in tree.iter(): print(elem)
print()

In [ ]:
counter = 0


tree = ET.parse('../../data/TeamScream.tcx')
# tree = ET.parse('example.xml')
xiter = tree.iter()

try: 
#{
    while True:
        xnode = next(xiter)
        schema, tagname = parsetag(xnode)
        print(f'{tagname} : {len(xnode)}')
        
#         if not schema: print(tagname)
#         else:  print(schema + " : " + tagname)

        print(tagname)
        if tagname == 'Trackpoint':
            insert_trackpoint(xiter, len(xnode))
            
        # Tmp loop breaker
        if counter > 200: break
        counter += 1
#}
except StopIteration: pass
    
#     tag = re.sub(r'^\{.+\}', '', elem.tag)
#     text = elem.text.rstrip() if elem.text else ''
#     print(f"Tag: {tag}, Value: {text}, Attrib: {elem.attrib}")


In [ ]:
tree = ET.parse('../../data/TeamScream.tcx')
# tree = ET.parse('../../data/45_min_HIIT_Hills_Ride_with_Sam_Yo.tcx')
# enode = tree.iter()
# print(enode)
# for elem in tree.iter():
#     print elem

counter = 0
for elem in tree.iter():
    if counter > 125: break
    tag = re.sub(r'^\{.+\}', '', elem.tag)
    text = elem.text.rstrip() if elem.text else ''
    print(f"Tag: {tag}, Value: {text}, Attrib: {elem.attrib}")
    counter += 1




In [ ]:
# for child in root:
#     print(child.tag, child.attrib)